In [20]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
import json
from tqdm.auto import tqdm

import minsearch

In [21]:
load_dotenv()

True

In [22]:
api_key=os.environ.get("OPENAI_API_KEY"),
client = OpenAI(api_key=api_key)

## Ingestion

In [23]:
df = pd.read_csv('../data/data.csv')
documents = df.to_dict(orient='records')

In [24]:
prompt_template = """
You emulate a user of our fitness assistant applications.
Formulate 5 questions this user might ask based on a provided exercise.
Make the questions specific
The record should contain the answer to the questions, and the questions should 
be complete and not too short. Use fewer words as possible from the record. 

The record:
exercise_name : {exercise_name}
type_of_activity : {type_of_activity}
type_of_equipment : {type_of_equipment}
body_part : {body_part}
type : {type}
muscle_groups_activated : {muscle_groups_activated}
instructions : {instructions}

Provide the output in parsable JSON without using code blocks:

{{ questions: ["question1", "question2", ..., "question5"]}}
""".strip()

In [25]:
print(prompt_template.format(**documents[0]))

You emulate a user of our fitness assistant applications.
Formulate 5 questions this user might ask based on a provided exercise.
Make the questions specific
The record should contain the answer to the questions, and the questions should 
be complete and not too short. Use fewer words as possible from the record. 

The record:
exercise_name : Push-Up
type_of_activity : Strength
type_of_equipment : Bodyweight
body_part : Upper Body
type : Push
muscle_groups_activated : Chest, Triceps, Shoulders, Core
instructions : Start in a plank position, lower your body until your chest nearly touches the floor, then push back up to the starting position.

Provide the output in parsable JSON without using code blocks:

{ questions: ["question1", "question2", ..., "question5"]}


In [26]:
documents[2]

{'id': 2,
 'exercise_name': 'Deadlift',
 'type_of_activity': 'Strength',
 'type_of_equipment': 'Barbell',
 'body_part': 'Lower Body',
 'type': 'Pull',
 'muscle_groups_activated': 'Hamstrings, Glutes, Back, Core',
 'instructions': 'Stand with feet hip-width apart, grip the barbell, lift it by straightening your legs and hips, then lower it back to the ground.'}

In [27]:
prompt = prompt_template.format(**documents[5])

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model ="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
questions = llm(prompt)

AuthenticationError: Error code: 401 - {'error': {'message': "Incorrect API key provided: ('sk-pro*************************************************************************************************************************************************************A',). You can find your API key at https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
json.loads(questions)

In [ ]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
results = {}

In [ ]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    print(questions_raw)
    questions = json.loads(questions_raw)
    results[doc_id] = questions["questions"]

In [ ]:
results